In [1]:
!pip install sentence_transformers
import json
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import numpy as np
import pandas as pd

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

from google.colab import drive
drive.mount('/content/drive')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
embedder = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embedder.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

## Using Threadpool

In [11]:
%%time
def get_embedding(article_pair):
  article_tags, article_id = article_pair
  tags_embeddings = [embedder.encode(tag) for tag in article_tags]  # Embed each tag separately
  aggregate_embedding = sum(tags_embeddings) / len(tags_embeddings)
  return article_id, list(aggregate_embedding)

def process_articles(db):
    results = {}  # Use a list to store all batches for simplicity in JSON output
    max_workers = 10
    batch_size = 100

    article_tags = []
    article_ids = []
    for article in db:
      article_tags.append(article['tags'])
      article_ids.append(article['id'])
    article_id_pairs = list(zip(article_tags, article_ids))

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for i in range(0, len(article_id_pairs), batch_size):
            current_batch = article_id_pairs[i:i+batch_size]
            print(f"Starting batch processing for articles {i+1} to {min(i+batch_size, len(article_id_pairs))}")
            futures = {executor.submit(get_embedding, pair): pair for pair in current_batch}
            for future in as_completed(futures):
                article_id, tags = future.result()
                results[article_id] = tags
    return results


CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.63 µs


In [12]:
with open('/content/drive/MyDrive/final_db.json', "r") as f:
  db = json.load(f)

In [13]:
%%time
new_tags = process_articles(db)

Starting batch processing for articles 1 to 100
Starting batch processing for articles 101 to 200
Starting batch processing for articles 201 to 300
Starting batch processing for articles 301 to 400
Starting batch processing for articles 401 to 500
Starting batch processing for articles 501 to 600
Starting batch processing for articles 601 to 700
Starting batch processing for articles 701 to 800
Starting batch processing for articles 801 to 900
Starting batch processing for articles 901 to 1000
Starting batch processing for articles 1001 to 1100
Starting batch processing for articles 1101 to 1200
Starting batch processing for articles 1201 to 1300
Starting batch processing for articles 1301 to 1400
Starting batch processing for articles 1401 to 1500
Starting batch processing for articles 1501 to 1600
Starting batch processing for articles 1601 to 1700
Starting batch processing for articles 1701 to 1800
Starting batch processing for articles 1801 to 1900
Starting batch processing for art

In [20]:
for el,v in new_tags.items():
  print(el,v)
  break

aksixz7uun2gkpss [0.015406889, 0.04966922, 0.009974902, -0.025631754, -0.02760107, 0.004751317, 0.008135494, 0.0012976034, 0.01480301, 0.00038268356, 0.037177473, -0.013022452, -0.013577011, -0.029204687, 0.009085863, -0.031750005, 0.022266654, 0.003672222, 0.020139337, -0.02757288, -0.03343215, 0.0064559504, 0.0069391765, 0.0042415294, 0.026385715, -0.012319344, 0.019825863, 0.016159713, -0.0107869245, 0.041883316, -0.007872521, -0.017581927, -0.01282692, -0.029177682, 1.6961682e-06, -0.0124268085, 0.012744165, -0.014387024, -0.0073585534, -0.004105371, -0.014830281, -0.007598918, -0.07578573, 0.012584464, -0.021891564, 0.014377087, 0.030086763, -0.0028006837, -0.015658155, 0.027152633, 0.003578997, 0.025094219, -0.05267432, -0.025278598, -0.019497724, -0.008457069, 0.020388512, 0.0061353804, -0.036408592, -0.009993833, 0.0059329127, 0.0016725231, -0.0071441405, -0.018231789, -0.0114612235, -0.00030151661, 0.0017472418, -0.00056628627, 0.038265992, 0.01111431, 0.050456528, -0.00539398

In [22]:
assigned = 0
for article in db:
    id = article['id']
    article['tags_embeddings'] = list(np.array(new_tags[id]).astype(np.float64))
    assigned += 1
if assigned == len(db):
    print("All tags have been assigned")
else:
    print("Not all tags have been assigned")

All tags have been assigned


In [24]:
#Check results
db[0]

{'id': 'nos7tzp7jprxlqxe',
 'Text': 'GENEVA – The remains of a climber discovered in the Swiss Alps in 2022 have been identified as those of a British mountaineer who went missing 52 years ago, local police said on Thursday.It is the latest in a series of discoveries of remains of long-missing climbers revealed as the Alps’ glaciers melt and recede because of global warming.The climber was reported missing in July 1971, but search teams at the time turned up nothing, said police in the canton of Valais, south-west Switzerland.Then on Aug 22, 2022, two climbers found human remains on the Chessjengletscher glacier near Saas-Fee, an Alpine village in the Saas Valley.It took a year to identify the person as experts worked their way through the case files of missing climbers.Finally, with the help of Interpol Manchester and the police in Scotland, a relative was found and a DNA sample allowed them to identify the British mountaineer, the Swiss police said in a statement.The climber was form

In [25]:
with open('/content/drive/MyDrive/final_db_with_tags.json', "w") as file:
    json.dump(db, file, indent=4)